In [1]:
# 데이터 분석을 위한 패키지
import numpy as np
import pandas as pd

In [2]:
# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import FinanceDataReader as fdr

In [4]:
import koreanize_matplotlib

In [5]:
import datetime
from dateutil.relativedelta import relativedelta

In [6]:
# 모든 컬럼,행을 출력하도록 설정
pd.set_option('display.max_columns', None) # None으로 설정하면 모든 컬럼 출력
pd.set_option('display.max_rows', None) #None으로 설정하면 모든 행 출력

In [7]:
kospi = fdr.StockListing('KOSPI')
kosdaq = fdr.StockListing('KOSDAQ')
etfs = fdr.StockListing('ETF/KR')

In [ ]:
kospi.head()

In [ ]:
kospi['Code'].tolist()

In [10]:
kospi.shape

(961, 17)

In [65]:
kosdaq['Market'].value_counts()

Market
KOSDAQ           1735
KOSDAQ GLOBAL      48
Name: count, dtype: int64

In [63]:
kosdaq.nunique()

Code           1783
ISU_CD         1783
Name           1783
Market            2
Dept              8
Close          1343
ChangeCode        6
Changes         308
ChagesRatio     724
Open           1280
High           1281
Low            1297
Volume         1692
Amount         1704
Marcap         1783
Stocks         1745
MarketId          1
dtype: int64

In [66]:
kospi.columns

Index(['Code', 'ISU_CD', 'Name', 'Market', 'Dept', 'Close', 'ChangeCode',
       'Changes', 'ChagesRatio', 'Open', 'High', 'Low', 'Volume', 'Amount',
       'Marcap', 'Stocks', 'MarketId'],
      dtype='object')

In [57]:
kosdaq.head(20)

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,196170,KR7196170005,알테오젠,KOSDAQ GLOBAL,기술성장기업부,296000,1,19000,6.86,280000,299500,275500,1769852,509107857000,15782373088000,53318828,KSQ
1,247540,KR7247540008,에코프로비엠,KOSDAQ GLOBAL,우량기업부,115300,1,500,0.44,115300,121000,114800,753672,88962834800,11276494963200,97801344,KSQ
2,028300,KR7028300002,HLB,KOSDAQ,중견기업부,66700,2,-900,-1.33,68000,69800,66300,867146,58297052800,8763541781100,131387433,KSQ
3,086520,KR7086520004,에코프로,KOSDAQ,우량기업부,61500,2,-1600,-2.54,63400,64500,61400,862807,54204237000,8188007910000,133138340,KSQ
4,141080,KR7141080002,리가켐바이오,KOSDAQ GLOBAL,기술성장기업부,103300,1,6000,6.17,98700,104700,96500,841696,85010962100,3781847915400,36610338,KSQ
5,145020,KR7145020004,휴젤,KOSDAQ GLOBAL,우량기업부,279500,3,0,0.00,278500,288000,277500,99772,28115379500,3522827223500,12604033,KSQ
6,000250,KR7000250001,삼천당제약,KOSDAQ,중견기업부,141000,2,-12500,-8.14,163900,166300,138800,2146239,318794485200,3307503552000,23457472,KSQ
7,214150,KR7214150005,클래시스,KOSDAQ GLOBAL,우량기업부,50100,1,1250,2.56,48900,50100,48900,141419,7009599400,3281833515900,65505659,KSQ
8,058470,KR7058470006,리노공업,KOSDAQ GLOBAL,우량기업부,200000,1,5600,2.88,195300,202500,194400,111132,22080537300,3048474000000,15242370,KSQ
9,277810,KR7277810008,레인보우로보틱스,KOSDAQ,기술성장기업부,145700,1,10900,8.09,135700,148700,134500,525056,75824554500,2826559310600,19399858,KSQ


In [20]:
kosdaq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1783 entries, 0 to 1782
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Code         1783 non-null   object 
 1   ISU_CD       1783 non-null   object 
 2   Name         1783 non-null   object 
 3   Market       1783 non-null   object 
 4   Dept         1783 non-null   object 
 5   Close        1783 non-null   object 
 6   ChangeCode   1783 non-null   object 
 7   Changes      1783 non-null   int64  
 8   ChagesRatio  1783 non-null   float64
 9   Open         1783 non-null   int64  
 10  High         1783 non-null   int64  
 11  Low          1783 non-null   int64  
 12  Volume       1783 non-null   int64  
 13  Amount       1783 non-null   int64  
 14  Marcap       1783 non-null   int64  
 15  Stocks       1783 non-null   int64  
 16  MarketId     1783 non-null   object 
dtypes: float64(1), int64(8), object(8)
memory usage: 236.9+ KB


In [49]:
# 날짜는 파이썬 표준라이브러리 datetime 사용
import datetime
# 오늘 날짜 구하기
today = datetime.datetime.today()
today_str = today.strftime('%Y-%m-%d')


# n개월 전 날짜 계산 함수(개월단위)
def calculate_start_date(months_ago, end_date):
    start_date = datetime.datetime.strptime(end_date, '%Y-%m-%d') - relativedelta(months=months_ago)
    return start_date.strftime('%Y-%m-%d')
  
# 시장과 기간을 입력받아, 모든 종목을 돌면서 수익률을 반환해주는 함수

def return_by_stock(market='KOSPI', month_ago = 1, end_date=today_str):
  
  start_date = calculate_start_date(month_ago, end_date)
  
  results = [] # 수익률 결과를 저장할 리스트
  
  market_list= fdr.StockListing(market)
  
  try:
    
    for stock in market_list['Code'].tolist():
      data = fdr.DataReader(stock, start=start_date, end=end_date)[['Close']]
      
      if data.empty:  # 데이터가 비어있는지 확인
        print(f"종목 {stock}의 데이터가 없습니다. 건너뜁니다.")
        continue
      
      # 일일 수익률
      data['return'] = data.pct_change()
      
      # NaN 값 처리: 첫 번째 행 삭제
      data = data.dropna()
      
      # 수익률을 퍼센트로 변환
      data['return'] = data['return'] * 100
      
      # 499790, 177900, 487360 코드에서 오류발생생
      # 데이터를 가져오는 중 오류가 발생했습니다: single positional indexer is out-of-bounds : 특정 인덱스에 접근할 때 그 인덱스가 존재하지 않는 경우
      # 상장된지 얼마 되지않아, 단 '하루'의 데이터만 존재하는경우, pct_change()를 호출하면 맨처음 NaN이 반환되고, dropna()를 호출하면 데이터프레임이 비게 됩니다.
      # 따라서 data가 비어있게 되어, 이후의 수익률 계산에서 single positional indexer is out-of-bounds 오류가 발생합니다. 이는 인덱스 접근을 시도할 때 데이터가 없기 때문에 발생합니다.
      
      if len(data) == 1:  # 상장이후 일일수익률 데이터가 단 하나라  수익률 계산이 의미가 없는경우
        print(f"종목 {stock}은 일일수익률 데이터가 하나라, 수익률 계산이 의미가 없기때문에 건너뜁니다.")
        continue
      
      if len(data) == 0: # 상장이후 종가가 하나라 수익률 계산 자체가 되지 않는경우
          print(f"종목 {stock}의 데이터가 충분하지 않습니다. 건너뜁니다.")
          continue
        
      # 해당 기간 전체 수익률 계산
      total_return = (data['Close'].iloc[-1] - data['Close'].iloc[0]) / data['Close'].iloc[0] * 100

      # 통계 정보 추가
      average_return = data['return'].mean()
      std_dev_return = data['return'].std()
      
      # 결과 저장
      results.append({
          '종목': stock,
          '총 수익률': total_return,
          '평균 수익률': average_return,
          '표준편차': std_dev_return
      })

      # # 결과 출력
      # print(f"종목: {stock}, {month_ago}개월 총 수익률: {total_return:.2f}%, 평균 수익률: {average_return:.2f}%, 표준편차: {std_dev_return:.4f}")
   

    return pd.DataFrame(results)
  except Exception as e:
    print(f"데이터를 가져오는 중 오류가 발생했습니다: {e}")
    return None


In [ ]:
# 종목 177900의 데이터가 충분하지 않습니다. 건너뜁니다.
# 종목 487360의 데이터가 충분하지 않습니다. 건너뜁니다.

In [68]:
# 각 시장의 평균 수익률과 표준편차를 계산하는 함수

def calculate_market_statistics(market='KOSPI', month_ago=1):
    results_df = return_by_stock(market, month_ago)
    if results_df is not None:
      
        # 평균 수익률 및 표준편차 계산
        average_return = results_df['총 수익률'].mean()
        std_dev_return = results_df['총 수익률'].std()
        return average_return, std_dev_return
    return None, None

# 시장별 통계 계산
kospi_avg_return, kospi_std_dev = calculate_market_statistics(market='KOSPI', month_ago=3)
kosdaq_avg_return, kosdaq_std_dev = calculate_market_statistics(market='KOSDAQ', month_ago=3)
etf_avg_return = etfs['EarningRate'].mean()
etf_std_dev = etfs['EarningRate'].std()

# 결과 출력

print("KOSPI 평균 수익률: {:.2f}%, 표준편차: {:.4f}".format(kospi_avg_return, kospi_std_dev))
print("KOSDAQ 평균 수익률: {:.2f}%, 표준편차: {:.4f}".format(kosdaq_avg_return, kosdaq_std_dev))
print("ETF 평균 수익률: {:.2f}%, 표준편차: {:.4f}".format(etf_avg_return, etf_std_dev))


종목 499790은 일일수익률 데이터가 하나라, 수익률 계산이 의미가 없기때문에 건너뜁니다.
종목 177900의 데이터가 충분하지 않습니다. 건너뜁니다.
종목 487360은 일일수익률 데이터가 하나라, 수익률 계산이 의미가 없기때문에 건너뜁니다.
KOSPI 평균 수익률: -4.91%, 표준편차: 20.6454
KOSDAQ 평균 수익률: -7.12%, 표준편차: 37.0179
ETF 평균 수익률: 1.73%, 표준편차: 14.3083


In [52]:
etfs.head()

,Symbol,Category,Name,Price,RiseFall,Change,ChangeRate,NAV,EarningRate,Volume,Amount,MarCap
0,459580,6,KODEX CD금리액티브(합성),1057920,2,95,0.01,1057938.0,0.8474,211396,223639,94381
1,360750,4,TIGER 미국S&P500,21665,2,100,0.46,21718.0,14.1524,3024886,65365,68331
2,357870,6,TIGER CD금리투자KIS(합성),55445,2,5,0.01,55444.0,0.8457,201715,11183,64632
3,069500,1,KODEX 200,32425,2,25,0.08,32457.0,-7.1317,2887122,93538,55982
4,133690,4,TIGER 미국나스닥100,139330,2,600,0.43,139699.0,18.0002,220590,30652,44627


In [55]:
# etfs['EarningRate'].mean()

etfs = fdr.StockListing('ETF/KR')
etfs['EarningRate'].mean()
etfs['EarningRate'].std()

14.308292983895667